In [1]:
# !pangolin czb_only.fasta -p

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
sns.set(style='whitegrid', font_scale=1.2)

from covidhub.config import Config
from covidhub.google.utils import get_secrets_manager_credentials
import covid_database.util as util
from covid_database import (
    init_db,
)
from covid_database import session_scope
from covid_database.queries.ngs_sample_tracking_query import czb_id_sample_tracking_query, czb_id_sample_tracking_query_by_project
import sqlalchemy
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
import covid_database.models.ngs_sample_tracking as ngs

db_uri = util.get_db_uri("cliahub/cliahub_rds_read_prod")
interface = init_db(util.get_db_uri("cliahub/cliahub_rds_read_prod"))
engine = create_engine(db_uri)
sm = sessionmaker(bind=engine)
session = sm()

In [3]:
## map from project RR code -> county name

def get_rr_county_map(session=session):
    fields_of_interest = (ngs.Project.location, ngs.Project.rr_project_id, )
    distinct_locations = (
        session.query(
            *fields_of_interest
        ).with_entities(
            *fields_of_interest
        )
    )
    df = pd.read_sql(
        distinct_locations.statement, 
        distinct_locations.session.bind
    )
    # return mapping of rr_project_id to location
    return {r['rr_project_id'] : str(r['location']) for idx, r in df.iterrows()}

rr_to_county = get_rr_county_map()

In [4]:
## Pull metadata for **all** samples with a CZB ID
all_meta = czb_id_sample_tracking_query(interface)[['gisaid_name', 'collection_date', 'czb_id', 'status', 'external_accession']]

## Tidy up formatting, map from projects -> counties
all_meta['gisaid_name'] = all_meta['gisaid_name'].map(lambda i: i[8:] if i!=None else np.nan) ## trim off hcov-19/ prefix
all_meta['RR'] = all_meta['czb_id'].map(lambda i: i.split('_')[0] if i!=None else i) ## trim off CZB ID
all_meta['county'] = all_meta['RR'].map(lambda i: str(rr_to_county[i]) if i in rr_to_county else np.nan) ## map back from RR to county

## fix weird dtypes because pandas and sql don't always play nicely
all_meta['status'] = all_meta['status'].map(lambda i: i.value if i!=None else '') ## fix enum type
all_meta.replace('NaN', np.nan, inplace=True) ## wtf with the dtypes, pandas
all_meta.replace(np.datetime64('NAT'), np.nan, inplace=True)

qc_pass_meta = all_meta.loc[all_meta['status'].str.contains("PASS", na=False)]

In [5]:
## import lineage report, pull out columns we want, rename index to match
lineages = pd.read_csv('./lineage_report.csv') 
lineages = lineages.loc[lineages['probability']>=0.9][['taxon', 'lineage']]
lineages.columns = ['gisaid_name', 'lineage'] 

## merge metadata + lineage info
linelist = qc_pass_meta.merge(lineages, on='gisaid_name', how='outer') ## attach lineage info
linelist = linelist[['gisaid_name', 'collection_date', 'county', 'lineage']] ## drop unnecessary columns
linelist.columns = ['sample', 'collection_date', 'county', 'lineage']

In [6]:
## Filter to samples with valid 'county' name, 'collection_date', and 'gisaid_name'
linelist = linelist.loc[
    (linelist['county'].notnull()) &    
    (linelist['sample'].notnull()) & 
    (linelist['collection_date'].notnull())
                           ]   
linelist['collection_date'] = pd.to_datetime(linelist['collection_date'], errors='coerce')

In [7]:
counts = linelist.groupby(['county', 'lineage', 
                           pd.Grouper(key='collection_date', freq='W-MON')]).agg('count').reset_index()
counts.rename(columns={'sample': 'count'}, inplace=True)
# counts['cumulative_count'] = counts['count'].cumsum()

In [8]:
print(linelist.head(),'\n\n')
print(counts.head())

linelist.to_csv('lineage_line_list.csv')
counts.to_csv('lineage_counts.csv')

                   sample collection_date              county    lineage
50  USA/CA-CZB-10198/2020      2020-05-12  Santa Clara County    B.1.503
51  USA/CA-CZB-10199/2020      2020-06-18  Santa Clara County        B.1
52  USA/CA-CZB-10202/2020      2020-05-11  Santa Clara County  B.1.1.119
53  USA/CA-CZB-10211/2020      2020-05-13  Santa Clara County    B.1.452
54  USA/CA-CZB-10212/2020      2020-05-11  Santa Clara County        B.1 


           county lineage collection_date  count
0  Alameda County     A.1      2020-04-06      2
1  Alameda County     A.1      2020-04-13      5
2  Alameda County     A.1      2020-04-20      1
3  Alameda County     A.1      2020-05-04      1
4  Alameda County     A.1      2020-05-18      1


In [8]:
# def plot_variants_over_time(counts=counts, variants=None):
    
#     counties = pd.unique(counts['county'])
#     fig, axes = plt.subplots(nrows=len(counties), ncols=1, sharex=True)

#     for county, ax in zip(counties, axes):
#         data = counts.loc[counts['county'] == county]
#         sns.lineplot(data=data, x='collection_date', y='sample', hue='lineage')
#         ax.set_ylabel(county)
#         ax.set_xlabel('Week (lagging)')
        
#     plt.tight_layout()
#     plt.show()
    
# plot_variants_over_time()